<!--NAVIGATION-->
<span style='background: rgb(128, 128, 128, .15); width: 100%; display: block; padding: 10px 0 10px 10px'>< [Series & Dataframes](04.01-Dataframes.ipynb) | [Contents](00.00-Index.ipynb) | [Quiz 4](04.03-Quiz.ipynb) ></span>

<a href="https://colab.research.google.com/github/eurostat/e-learning/blob/main/python-official-statistics/04.02-Pandas.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<a id='top'></a>

# Pandas: Advanced Processing
## Content  
- [Load from sources (and save to)](#load)
- [Handling missing data](#missing)
- [Dealing with duplicates](#duplicates)
- [Hierarchical Indexing](#hierarchical)
- [Combining Datasets: Concat, Append, Merge and Join](#combining)
- [Aggregation and Grouping](#aggregation)
- [Pivot Tables](#pivot)
- [Time Series](#time)

<a id='load'></a>
 
## Load from sources (and save to)
Usually data to process comes from all kind of sources in a variety of formats.
Pandas provides helper functions to read data from various file formats like CSV, Excel spreadsheets, HTML tables, JSON, SQL, and more. And the same capabilities for saving dataframes to different formats. We'll use a little bit of Jupyter magic to help you find them:

In [ ]:
import pandas as pd
# magic for read functions
?pd.read_*

In [ ]:
# magic for save functions
A = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
?A.to_*

For exemplification we saved an Eurostat database `hlth_ehis_pe9e.tsv.gz` in our local folder data. This file is TAB separated and also archived, but not a problem for Pandas:

In [ ]:
import numpy as np
health = pd.read_csv('data/hlth_ehis_pe9e.tsv.gz', sep='\t', index_col=0)
print(health.describe())
# play with column names (because of spaces)
print(health.columns)
health.columns = map(str.strip, list(health.columns.array))
print(health.columns)

print(health.head())

And now we will save the column 'BE' as a table into a SQLite database. As simple as two lines of code:

In [ ]:
import sqlite3
with sqlite3.connect('data/test.db') as conn:
    health.to_sql('BE', conn, if_exists='replace')
# health.to_csv('data/test.csv.zip', compression='zip')
# health2 = pd.read_csv('data/test.csv.zip', index_col=0)
# print(health2.info())
# print(health.info())

<a id='missing'></a>
 
## Handling missing data
What does "missing data" mean? What is a missing value? It depends on the origin of the data and the context it was generated. For example, for a survey, a _`Salary`_ field with an empty value, or a number 0, or an invalid value (a string for example) can be considered "missing data". These concepts are related to the values that Python will consider "Falsy":


In [ ]:
falsy_values = (0, False, None, '', [], {})
any(falsy_values)


Numpy has a special "nullable" value for numbers which is `np.nan`. It's _NaN_: "Not a number".  
The `np.nan` value is kind of a virus. Everything that it touches becomes `np.nan`:

In [ ]:
print(3 + np.nan)
a = np.array([1, 2, 3, np.nan, np.inf, 4])
print(a.sum())
print(a.mean())

Numpy also supports an "Infinite" type: `np.inf`  
Which also behaves as a virus:

In [ ]:
print(3 + np.inf)
print(np.inf / 3)
print(np.inf / np.inf)

First example of processing missing data, for out `health` DataFrame, is trying to convert data to numeric, but this is not always possible, and will end up in some `nan` values.  
For a column (Series) the `to_numeric` function can be directly used. But this will not work for several columns or whole DataFrame. In this case we'll `apply` the function `to_numeric` for our columns: 

In [ ]:
# convert to float (handling the missing data)
# entyre dataframe
print(health.apply(pd.to_numeric, errors='coerce').head())
# selection of columns
print(health[['BE', 'RO']].apply(pd.to_numeric, errors='coerce').head())
# one column
print(pd.to_numeric(health['BE'], errors='coerce').head())

<a id='duplicates'></a>
 
## Dealing with duplicates
Checking duplicate values is extremely simple. Itll behave differently between `Series` and `DataFrames`. Let's start with Series. As an example, let's say United States is throwing a fancy party and are inviting Ambassadors from Europe. But can only invite one ambassador per country. This is our original list, and as you can see, both the UK and Germany have duplicated ambassadors:

In [ ]:
ambassadors = pd.Series([
    'France',
    'United Kingdom',
    'United Kingdom',
    'Italy',
    'Germany',
    'Germany',
    'Germany',
], index=[
    'Gérard Araud',
    'Kim Darroch',
    'Peter Westmacott',
    'Armando Varricchio',
    'Peter Wittig',
    'Peter Ammon',
    'Klaus Scharioth '
])
ambassadors

In [ ]:
# The two most important methods to deal with duplicates are `duplicated` (that will tell you which values are duplicates) and `drop_duplicates` (which will just get rid of duplicates):
print(ambassadors.duplicated())
# In this case `duplicated` didn't consider `'Kim Darroch'`, the first instance of the United Kingdom or `'Peter Wittig'` as duplicates. That's because, by default, it'll consider the first occurrence of the value as not-duplicate. You can change this behavior with the `keep` parameter:
print(ambassadors.duplicated(keep='last'))
# You can also choose to mark all of them as duplicates with `keep=False`:
print(ambassadors.duplicated(keep=False))
# A similar method is `drop_duplicates`, which just excludes the duplicated values and also accepts the `keep` parameter:
print(ambassadors.drop_duplicates())
print(ambassadors.drop_duplicates(keep='last'))
print(ambassadors.drop_duplicates(keep=False))

### Duplicates in DataFrames
Conceptually speaking, duplicates in a DataFrame happen at "row" level. Two rows with exactly the same values are considered to be duplicates:

In [ ]:
players = pd.DataFrame({
    'Name': [
        'Kobe Bryant',
        'LeBron James',
        'Kobe Bryant',
        'Carmelo Anthony',
        'Kobe Bryant',
    ],
    'Pos': [
        'SG',
        'SF',
        'SG',
        'SF',
        'SF'
    ]
})
players

In [ ]:
# In the previous DataFrame, we clearly see that Kobe is duplicated; but he appears with two different positions. What does `duplicated` say?
players.duplicated()
# Again, conceptually, "duplicated" means "all the column values should be duplicates". We can customize this with the `subset` parameter:
players.duplicated(subset=['Name'])
# And the same rules of `keep` still apply:
players.duplicated(subset=['Name'], keep='last')
# `drop_duplicates` takes the same parameters:
players.drop_duplicates()
players.drop_duplicates(subset=['Name'])
players.drop_duplicates(subset=['Name'], keep='last')

<a id='hierarchical'></a>
 
## Hierarchical Indexing
Up to this point we've been focused primarily on one-dimensional and two-dimensional data, stored in Pandas ``Series`` and ``DataFrame`` objects. Often it is useful to go beyond this and store higher-dimensional data–that is, data indexed by more than one or two keys.  
A common pattern in practice is to make use of *hierarchical indexing* (also known as *multi-indexing*) to incorporate multiple index *levels* within a single index.
In this way, higher-dimensional data can be compactly represented within the familiar one-dimensional ``Series`` and two-dimensional ``DataFrame`` objects.  
In a ``DataFrame``, the rows and columns are completely symmetric, and just as the rows can have multiple levels of indices, the columns can have multiple levels as well.  
In our example the multi-index reffers to columns:

In [ ]:
import pandas_datareader.data as web
df = web.DataReader('ttr00002', 'eurostat', start='2008')

In [ ]:
print(df.columns.names)
df = df.reorder_levels(['UNIT', 'FREQ', 'TRA_INFR', 'GEO'], axis=1, )
print(df.columns.get_level_values(0).unique())
print(df.columns.get_level_values(1).unique())
print(df.columns.get_level_values(2).unique())
df['Kilometre', 'Annual','Motorways']

<a id='combining'></a>
 
## Combining Datasets: Concat, Append, Merge and Join
### pd.concat()
Pandas has a function, ``pd.concat()``, which has a similar syntax to ``np.concatenate`` but contains a number of options that we'll discuss momentarily:

In [ ]:
# for series
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
print(ser1)
print(ser2)
print(pd.concat([ser1, ser2]))

In [ ]:
# Some simple helpers to speed up the examples
# Just a class used to display inline
class display(object):
	template = """<div style="float: left; padding: 10px;">
	<p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
	</div>"""
	def __init__(self, *args):
		self.args = args
	def _repr_html_(self):
		return '\n'.join(self.template.format(a, eval(a)._repr_html_()) for a in self.args)
	def __repr__(self):
		scope = locals()
		return '\n\n'.join(a + '\n' + repr(eval(a)) for a in self.args)
# create simple dataframes
def make_df(cols, ind):
	"""Quickly make a DataFrame"""
	data = {c: [str(c) + str(i) for i in ind]
			for c in cols}
	return pd.DataFrame(data, ind)

In [ ]:
# simple concatenation (by row)
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

In [ ]:
# simple concatenation (by col)
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis=1)")

In [ ]:
# duplicate index if not a problem
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y])')

In [ ]:
# or verify integrity of the index
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

In [ ]:
# or just ignoring the index
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

In [ ]:
# or create a multi-index
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

In [ ]:
# joining for columns that differ (all)
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

In [ ]:
# joining for common columns
display('df5', 'df6', "pd.concat([df5, df6], join='inner')")

### pd.merge & pd.join
One essential feature offered by Pandas is its high-performance, in-memory join and merge operations.
If you have ever worked with databases, you should be familiar with this type of data interaction.

In [ ]:
# One-to-one joins
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
df3 = pd.merge(df1, df2)
display('df1', 'df2', 'pd.merge(df1, df2)')

In [ ]:
# Many-to-one joins
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

In [ ]:
# Many-to-many joins
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

In [ ]:
# the same thing with join (but a key must be in place)
df1.set_index('employee', inplace=True)
df2.set_index('employee', inplace=True)
display('df1', 'df2', 'df1.join(df2)')

<a id='aggregation'></a>
 
## Aggregation and Grouping
For a ``DataFrame``, by default the aggregates return results within each column, or row:

In [ ]:
rng = np.random.RandomState(42)
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
print(df)
# by rows
print(df.sum())
# by columns
print(df.sum(axis='columns'))

Simple aggregations can give you a flavor of your dataset, but often we would prefer to aggregate conditionally on some label or index: this is implemented in the so-called ``groupby`` operation.
The name "group by" comes from a command in the SQL database language.

In [ ]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
display('df', 'df.groupby("key").sum()')

<a id='pivot'></a>
 
## Pivot Tables
The pivot table takes simple column-wise data as input, and groups the entries into a two-dimensional table that provides a multidimensional summarization of the data.
The difference between pivot tables and ``GroupBy`` can sometimes cause confusion; it helps me to think of pivot tables as essentially a *multidimensional* version of ``GroupBy`` aggregation.

In [ ]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.head()

In [ ]:
# with groupby
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

In [ ]:
# with pivot table
titanic.pivot_table('survived', index='sex', columns='class')

<a id='time'></a>
 
## Time Series
Time series are very well represented in Pandas, and we just scratch the surface here. For more information you can refer to the ["Time Series/Date" section](http://pandas.pydata.org/pandas-docs/stable/timeseries.html) of the Pandas online documentation. All starts with the elementary data types used for indexes:
### Python date and time
Python's basic objects for working with dates and times reside in the built-in [datetime](https://docs.python.org/3/library/datetime.html) module.
Along with the third-party [dateutil](http://labix.org/python-dateutil) module, you can use it to quickly perform a host of useful functionalities on dates and times.  
The power of ``datetime`` and ``dateutil`` lie in their flexibility and easy syntax: you can use these objects and their built-in methods to easily perform nearly any operation you might be interested in.
But they are not efficient to work with large arrays of dates and times:
just as lists of Python numerical variables are suboptimal compared to NumPy-style typed numerical arrays, lists of Python datetime objects are suboptimal compared to typed arrays of encoded dates.
### Typed arrays of times: NumPy's ``datetime64``
With this NumPy type you can create uniform arrays, where processing can be accomplished much more quickly. The trade-off is the number of units encoded $2^{64}$.  
For example, if you want a time resolution of one nanosecond, you only have enough information to encode a range of $2^{64}$ nanoseconds, or just under 600 years.
NumPy will infer the desired unit from the input; for example, here is a day-based datetime:

In [ ]:
print(np.datetime64('2015-07-04').dtype)
print(np.datetime64('2015-07-04 12:00').dtype)
print(np.datetime64('2015-07-04 12:59:59.50', 'ns').dtype)

The following table, drawn from the [NumPy datetime64 documentation](http://docs.scipy.org/doc/numpy/reference/arrays.datetime.html), lists the available format codes along with the relative and absolute timespans that they can encode:

|Code    | Meaning     | Time span (relative) | Time span (absolute)   |
|--------|-------------|----------------------|------------------------|
| ``Y``  | Year	       | ± 9.2e18 years       | [9.2e18 BC, 9.2e18 AD] |
| ``M``  | Month       | ± 7.6e17 years       | [7.6e17 BC, 7.6e17 AD] |
| ``W``  | Week	       | ± 1.7e17 years       | [1.7e17 BC, 1.7e17 AD] |
| ``D``  | Day         | ± 2.5e16 years       | [2.5e16 BC, 2.5e16 AD] |
| ``h``  | Hour        | ± 1.0e15 years       | [1.0e15 BC, 1.0e15 AD] |
| ``m``  | Minute      | ± 1.7e13 years       | [1.7e13 BC, 1.7e13 AD] |
| ``s``  | Second      | ± 2.9e12 years       | [ 2.9e9 BC, 2.9e9 AD]  |
| ``ms`` | Millisecond | ± 2.9e9 years        | [ 2.9e6 BC, 2.9e6 AD]  |
| ``us`` | Microsecond | ± 2.9e6 years        | [290301 BC, 294241 AD] |
| ``ns`` | Nanosecond  | ± 292 years          | [ 1678 AD, 2262 AD]    |
| ``ps`` | Picosecond  | ± 106 days           | [ 1969 AD, 1970 AD]    |
| ``fs`` | Femtosecond | ± 2.6 hours          | [ 1969 AD, 1970 AD]    |
| ``as`` | Attosecond  | ± 9.2 seconds        | [ 1969 AD, 1970 AD]    |

### Dates and times in pandas: best of both worlds
Pandas builds upon all the tools just discussed to provide a ``Timestamp`` object, which combines the ease-of-use of ``datetime`` and ``dateutil`` with the efficient storage and vectorized interface of ``numpy.datetime64``.

In [ ]:
# flexibility of Python
print(date := pd.to_datetime("4th of July, 2015"))
print(date.strftime('%A'))

# power of NumPy vectorization
print(date + pd.to_timedelta(np.arange(12), 'D'))

### Pandas Time Series: Indexing by Time

Where the Pandas time series tools really become useful is when you begin to *index data by timestamps*.
For example, we can construct a ``Series`` object that has time indexed data:

In [ ]:
index = pd.DatetimeIndex(['2014-07-04', '2014-08-04',
                          '2015-07-04', '2015-08-04'])
data = pd.Series([0, 1, 2, 3], index=index)
print(data)

# slicing
print(data['2014-07-04':'2015-07-04'])
print(data['2015'])


### Pandas Time Series Data Structures

This section will introduce the fundamental Pandas data structures for working with time series data:

- For *time stamps*, Pandas provides the ``Timestamp`` type. As mentioned before, it is essentially a replacement for Python's native ``datetime``, but is based on the more efficient ``numpy.datetime64`` data type. The associated Index structure is ``DatetimeIndex``.
- For *time Periods*, Pandas provides the ``Period`` type. This encodes a fixed-frequency interval based on ``numpy.datetime64``. The associated index structure is ``PeriodIndex``.
- For *time deltas* or *durations*, Pandas provides the ``Timedelta`` type. ``Timedelta`` is a more efficient replacement for Python's native ``datetime.timedelta`` type, and is based on ``numpy.timedelta64``. The associated index structure is ``TimedeltaIndex``.

In [ ]:
# DatetimeIndex creation: parsing available for a large variety of formats
import datetime
dates = pd.to_datetime([datetime.datetime(2015, 7, 3), '4th of July, 2015',
                       '2015-Jul-6', '07-07-2015', '20150708'])
print(dates)
# or pd.date_range
print(pd.date_range('2015-07-03', '2015-07-10'))

# PeriodIndex from DatetimeIndex
periods = dates.to_period('D')
print(periods)
# or pd.period_range
print(pd.period_range('2015-07', periods=8, freq='M'))

# TimedeltaIndex is created, for example, when a date is subtracted from another
print(dates - dates[0])
# or pd.timedelta_range
print(pd.timedelta_range(0, periods=10, freq='H'))

### Frequencies and Offsets

Fundamental to these Pandas time series tools is the concept of a frequency or date offset.
Just as we saw the ``D`` (day) and ``H`` (hour) codes above, we can use such codes to specify any desired frequency spacing.
The following table summarizes the main codes available:

| Code   | Description         | Code   | Description          |
|--------|---------------------|--------|----------------------|
| ``D``  | Calendar day        | ``B``  | Business day         |
| ``W``  | Weekly              |        |                      |
| ``M``  | Month end           | ``BM`` | Business month end   |
| ``Q``  | Quarter end         | ``BQ`` | Business quarter end |
| ``A``  | Year end            | ``BA`` | Business year end    |
| ``H``  | Hours               | ``BH`` | Business hours       |
| ``T``  | Minutes             |        |                      |
| ``S``  | Seconds             |        |                      |
| ``L``  | Milliseonds         |        |                      |
| ``U``  | Microseconds        |        |                      |
| ``N``  | nanoseconds         |        |                      |


### Frequency conversion
The primary function for changing frequencies is the asfreq() method. For a DatetimeIndex, this is basically just a thin, but convenient wrapper around reindex() which generates a date_range and calls reindex.

In [ ]:
dr = pd.date_range("1/1/2010", periods=3, freq=3 * pd.offsets.BDay())
ts = pd.Series(np.random.randn(3), index=dr)
print(ts)
print(ts.asfreq(pd.offsets.BDay())
)

### Resampling

pandas has a simple, powerful, and efficient functionality for performing resampling operations during frequency conversion (e.g., converting secondly data into 5-minutely data).

resample() is a time-based groupby, followed by a reduction method on each of its groups.

In [ ]:
# downsampling
rng = pd.date_range("1/1/2012", periods=300, freq="S")
ts = pd.Series(np.random.randint(0, 500, len(rng)), index=rng)
ts.resample("1Min").sum()

In [ ]:
# upsampling
# from secondly to every 250 milliseconds
print(ts[:2].resample("250L").asfreq())
print(ts[:2].resample("250L").ffill())
print(ts[:2].resample("250L").ffill(limit=2))

<!--NAVIGATION-->
<span style='background: rgb(128, 128, 128, .15); width: 100%; display: block; padding: 10px 0 10px 10px'>< [Series & Dataframes](04.01-Dataframes.ipynb) | [Contents](00.00-Index.ipynb) | [Quiz 4](04.03-Quiz.ipynb) > [Top](#top) ^ </span>

<span style='background: rgb(128, 128, 128, .15); width: 100%; display: block; padding: 10px 0 10px 10px'>This is the Jupyter notebook version of the __Python for Official Statistics__ produced by Eurostat; the content is available [on GitHub](https://github.com/eurostat/e-learning/tree/main/python-official-statistics).
<br>The text and code are released under the [EUPL-1.2 license](https://github.com/eurostat/e-learning/blob/main/LICENSE).</span>